# Aggregate functions
Aggregate functions operate on groups of tiles at once. An example of this type of function is finding the mean cell values of a column of tiles. Other statistical and diagnostic tasks can be performed on columns of tiles at a time.

In [1]:
import astraea.spark.rasterframes._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import geotrellis.spark.SpatialKey._
import geotrellis.raster._
import geotrellis.raster.mapalgebra.local._


implicit val spark = SparkSession.builder().
  master("local").appName("RasterFrames").
  config("spark.ui.enabled", "false").
  getOrCreate().
  withRasterFrames

val sc = spark.sparkContext

def readTiff(name: String): SinglebandGeoTiff = SinglebandGeoTiff(s"../samples/$name")
val filenamePattern = "L8-B%d-Elkton-VA.tiff"

Intitializing Scala interpreter ...

Spark Web UI available at http://172.18.0.2:4041
SparkContext available as 'sc' (version = 2.2.0, master = local[*], app id = local-1528922422958)
SparkSession available as 'spark'


import astraea.spark.rasterframes._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import geotrellis.spark.SpatialKey._
import geotrellis.raster._
import geotrellis.raster.mapalgebra.local._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@33770d29
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@8ce2ec
readTiff: (name: String)geotrellis.raster.io.geotiff.SinglebandGeoTiff
filenamePattern: String = L8-B%d-Elkton-VA.tiff


In [2]:
def bandsToJoin(bands: Seq[Int]) = bands.
  map { b ⇒ (b, filenamePattern.format(b)) }.
  map { case (b, f) ⇒ ((b - 1) % 3, readTiff(f)) }.
  map { case (b, t) ⇒ t.projectedRaster.toRF(s"col_$b") }.
  reduce(_ spatialJoin _)

bandsToJoin: (bands: Seq[Int])astraea.spark.rasterframes.RasterFrame


Create a contrived example of columns with multiple tiles. Bands 1,4,7 are in col_0, 2,5,8 are in col_1, and 3,6,9 are in col_2.

In [3]:
val bands1 = bandsToJoin(Seq(1,2,3))
val bands2 = bandsToJoin(Seq(4,5,6))
val bands3 = bandsToJoin(Seq(7,8,9))
val grouped = bands1.union(bands2).union(bands3)

bands1: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, col_0: rf_tile ... 2 more fields]
bands2: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, col_0: rf_tile ... 2 more fields]
bands3: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, col_0: rf_tile ... 2 more fields]
grouped: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [spatial_key: struct<col: int, row: int>, col_0: rf_tile ... 2 more fields]


Compute statistics based on aggregates of tiles. These are all column-wise operations, so `aggMean` computes the average of all cells of all tiles in the first column.

In [ ]:
grouped.select(aggMean($"col_0")).show()

`aggDataCells` returns the number of cells in a column that are no NODATA. `aggNoDataCells` does the exact opposite, returning the numbers of NODATA cells.

In [ ]:
grouped.select(aggDataCells($"col_1")).show()

## Local aggregate functions
Local functions operate on a cell-by-cell basis. For instance, `localAggMax` examines every tile in a column and the output cells are each the maximum of the number (in this case, three) of values that it sees. The output is a tile with every cell as the maximum of the corresponding cells 

In [4]:
import geotrellis.raster._
val array1: Array[Int] = (1 to 9).toArray
val tile1: Tile = IntArrayTile(array1, 3, 3)
val array2: Array[Int] = (9 to 1 by -1).toArray
val tile2: Tile = IntArrayTile(array2, 3, 3)
val array3: Array[Int] = Array.fill(9)(5)
val tile3: Tile = IntArrayTile(array3, 3, 3)

import geotrellis.raster._
array1: Array[Int] = Array(1, 2, 3, 4, 5, 6, 7, 8, 9)
tile1: geotrellis.raster.Tile = IntConstantNoDataArrayTile([I@2f4ec8e2,3,3)
array2: Array[Int] = Array(9, 8, 7, 6, 5, 4, 3, 2, 1)
tile2: geotrellis.raster.Tile = IntConstantNoDataArrayTile([I@418610f3,3,3)
array3: Array[Int] = Array(5, 5, 5, 5, 5, 5, 5, 5, 5)
tile3: geotrellis.raster.Tile = IntConstantNoDataArrayTile([I@b38595f,3,3)


In [5]:
val df = sc.parallelize(Seq(tile1, tile2, tile3)).toDF("tiles")

df: org.apache.spark.sql.DataFrame = [tiles: rf_tile]


For instance, since each tile is filled with data cells, the result of aggregating a count of them is that each cell contains three, one for each tile aggregated over. `localAggNoDataCells` works in the same way.

In [75]:
df.select(tileHistogram(localAggDataCells($"tiles"))).show(false)

+-------------------+---------+
|stats              |bins     |
+-------------------+---------+
|[9,3.0,3.0,3.0,0.0]|[[3.0,9]]|
+-------------------+---------+



Functions also exist for finding the aggregated max, mean, and min

In [ ]:
val showTile = udf((t: Tile) => t.asciiDraw())
df.select(showTile(localAggMax($"tiles"))).show(false)
df.select(showTile(localAggMean($"tiles"))).show(false)

`localAggStats` will return a structure with columns representing a variety of different tile-wise aggregate statistics.

In [6]:
df.select(localAggStats($"tiles") as "stats").select("stats.*").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|               count|                 min|                 max|                mean|            variance|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|IntUserDefinedNoD...|IntConstantNoData...|IntConstantNoData...|DoubleConstantNoD...|DoubleConstantNoD...|
+--------------------+--------------------+--------------------+--------------------+--------------------+

